# Objaverse

Objaverse is a massive dataset of annotated 3D objects.

It is hosted on 🤗[Hugging Face](https://huggingface.co/datasets/allenai/objaverse). This notebook provides a tutorial on downloading objects and annotations!

We'll get started by downloading and importing the `objaverse` package from PyPi, which will allow us to easily download subsets of the dataset:

In [ ]:
!pip install --upgrade --quiet objaverse

import objaverse
objaverse.__version__

# Loading UIDs

Each object has a unique corresponding UID (universal identifier). Let's get all the object UIDs:

In [ ]:
uids = objaverse.load_uids()
len(uids), type(uids)

Here, `uids` is a list of strings. Let's look at a few of them:

In [ ]:
uids[:10]

# Loading Annotations

We can get the object annotations for each of object using the
```python
objaverse.load_annotations(
    uids: Optional[List[str]] = None
) -> Dict[str, Dict[str, Any]]
```

function. The function optionally takes in a list of the `uids` and returns a map of each specified uid to its corresponding annotations. If `uids` is not specified, the annotations for every object is returned.

In [ ]:
annotations = objaverse.load_annotations(uids[:10])
annotations

Now, if we want to look at a particular UID's annotation, we can do:

In [ ]:
annotations[uids[0]]

Here, we see properties like the "name", "license", "description", "tags", and other metadata.

### Filtering

We can use the annotations to filter for particular properties of objects. For example, we can filter for objects that are distributed with the CC-BY license:

In [ ]:
annotations = objaverse.load_annotations()

> Note: Loading all annotations for the first time may take a few minutes. The annotations are being downloaded and saved locally.

And then use a list comprehension to get the CC-BY licensed objects

In [ ]:
cc_by_uids = [uid for uid, annotation in annotations.items() if annotation["license"] == "by"]
cc_by_uids[:10]

# Downloading Objects

We're going to use multiprocessing to download the objects. First, let's check our CPU count to get the number of processes we'll use:

In [ ]:
import multiprocessing
processes = multiprocessing.cpu_count()
processes

For ease of demonstration, let's randomly sample 100 objects that we'll want to download:

In [ ]:
import random

random.seed(42)

uids = objaverse.load_uids()
random_object_uids = random.sample(uids, 10)

random_object_uids

And now, let's download them with `objaverse.load_objects` with all of our processes. The function

```python
objaverse.load_objects(
    uids: List[str],
    download_processes: int = 1
) -> Dict[str, str]
```

takes in a list of object UIDs and optionally the number of download processes, and returns a map from each object UID to its `.glb` file location on disk:

In [ ]:
objects = objaverse.load_objects(
    uids=random_object_uids,
    download_processes=processes
)
objects

Once objects are downloaded locally for the first time, subsequent calls to `load_objects` are cached, and thus much faster:

In [ ]:
objects = objaverse.load_objects(uids=random_object_uids)

Let's load up one of the `.glb` files to visualize it:

In [ ]:
!pip install trimesh --quiet

In [ ]:
import trimesh
trimesh.load(list(objects.values())[0]).show()

> **NOTE: I highly recommend using Blender for rendering. It will texture the meshes much better than trimesh. I just wanted to show a quick visualization.**

# LVIS Annotations

Finally, we'll load in the LVIS annotations, which returns a dictionary that maps from the LVIS category (key) to a list of object UIDs within that category (value):

In [ ]:
lvis_annotations = objaverse.load_lvis_annotations()
lvis_annotations

Note, too, that we have currently removed and are reviewing a few thousand of the objects that were detected as scans of humans to allow other research organizations to use the dataset.

For more information, please visit our [website](objaverse.allenai.org) or open a discussion on our 🤗[Hugging Face repo](https://huggingface.co/datasets/allenai/objaverse/discussions).